In [ ]:
import pandas as pd
import numpy as np
import os
import sqlite3
import shutil
import datetime
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from IPython.display import HTML, display, Markdown
import tabulate
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
import graphviz
from GraphVizUtil import *
from GraphVizFormats import *
import warnings
warnings.filterwarnings('ignore')
import seaborn as sb
sb.set(style='darkgrid', font_scale=1.2)

def filter_descriptions(tech_comm_desc):
    try:
        tech_comm_desc = tech_comm_desc.values[0][0].replace('#', '').replace('"','').replace("\n",'').strip()
    except:
        tech_comm_desc = 'No description provided'
    return tech_comm_desc

def create_args_flowd(df_graph):
    nodes, tech, ltech, to_tech, from_tech = set(), set(), set(), set(), set()
    for ind,row in df_graph.iterrows():
        #descriptions:
        input_comm_des = filter_descriptions(pd.read_sql("SELECT comm_desc FROM commodities WHERE comm_name='" + row['input_comm'] + "'", con))
        output_comm_des = filter_descriptions(pd.read_sql("SELECT comm_desc FROM commodities WHERE comm_name='" + row['output_comm'] + "'", con))
        tech_des = filter_descriptions(pd.read_sql("SELECT tech_desc FROM technologies WHERE tech='" + row['tech'] + "'", con))

        if 'ethos' in row['input_comm']:
            ltech.add('"' + row['tech'] + '"' +  ' [tooltip = "' + tech_des + '"]')
        else :
            nodes.add('"' + row['input_comm'] + '"' +  ' [tooltip = "' + input_comm_des + '"]')
        nodes.add('"' + row['output_comm'] + '"' +  ' [tooltip = "' + output_comm_des + '"]')
        tech.add('"' + row['tech'] + '"' +  ' [tooltip = "' + tech_des + '"]')

        if row['input_comm'] != 'ethos':
            to_tech.add('"%s"' % row['input_comm'] + '\t->\t"%s"' % row['tech']) 
        from_tech.add('"%s"' % row['tech'] + '\t->\t"%s"' % row['output_comm'])
    args = dict(
    enodes = "".join('%s;\n\t\t' % x for x in nodes),
    tnodes = "".join('%s;\n\t\t' % x for x in tech),
    iedges = "".join('%s;\n\t\t' % x for x in to_tech),
    oedges = "".join('%s;\n\t\t' % x for x in from_tech),
    snodes = ";".join('%s' %x for x in ltech),
    )
    return args

def return_format_colors():
    colors = {}
    colors.update(getColorConfig(False))
    return colors, quick_run_dot_fmt

def return_flowd_table(final_dem, level=1):
    df = pd.read_sql("SELECT * FROM Efficiency", con)
    df_sel = df[df['output_comm']==final_dem]
    if len(df_sel)==0:
        df_sel = df[df['tech']==final_dem]
    inputs = df_sel['input_comm'].unique()
    iterval=0
    if level!=0:
        while len(inputs)>0:
            df_append = df[df['output_comm'].isin(inputs)]
            df_sel = pd.concat([df_sel, df_append])
            inputs = df_append['input_comm'].unique()
            iterval+=1
            if iterval>level-1:
                break
    df_graph = df_sel[['input_comm', 'tech', 'output_comm']].drop_duplicates()
    return df_graph

def return_flowd_table_fwds(final_dem):
    df = pd.read_sql("SELECT * FROM Efficiency", con)
    df_sel = df[df['output_comm']==final_dem]
    if len(df_sel)==0:
        df_sel = df[df['tech']==final_dem]
    inputs = df_sel['input_comm'].unique()
    outputs = df_sel['output_comm'].unique()

    iterval=0
    while len(inputs)>0:
        df_append = df[df['output_comm'].isin(inputs)]
        df_sel = pd.concat([df_sel, df_append])
        inputs = df_append['input_comm'].unique()
        iterval+=1
        if iterval>2:
            break
    iterval=0
    while len(outputs)>0:
        df_append = df[df['input_comm'].isin(outputs)]
        df_sel = pd.concat([df_sel, df_append])
        outputs = df_append['output_comm'].unique()
        iterval+=1
        if iterval>=0:
            break

    df_graph = df_sel[['input_comm', 'tech', 'output_comm']].drop_duplicates()
    return df_graph

con = sqlite3.connect(r'../US_9R_8D.sqlite') #change path to database
cur = con.cursor()   
con.text_factory = str 

def controls_rows(w):
    controls = HBox(w.children[:-1], layout = Layout(flex_flow='row wrap', width='max-content'))
    output = w.children[-1]
    display(VBox([controls, output],  layout = Layout(flex_flow='columns wrap', width='max-content', size=10)))
    
#https://www.eia.gov/energyexplained/units-and-calculators/
btu_per_oil_bbl = 5698000
PJ_per_oil_bbl = btu_per_oil_bbl*1.05506e-12

### Overview
This notebook describes on a series of policy scenarios in the [OEO 2022 report](https://www.cmu.edu/energy/key-initiatives/open-energy-outlook/oeo-report-2022.html) that highlight the interplay between policy and technology to achieve deep reductions in CO<sub>2</sub> emissions by 2050. The scenarios described below are meant to cover a wide range of plausible outcomes that lead to varying degrees of emissions reduction.

### No new policy beyond 2021 (‘No Policy’)
This scenario assumes that all existing policies as of the end of 2021 remain in their current form, and no new federal or state policies are implemented. The results indicate how projected fuel prices and technology costs will shape the energy system in the absence of climate policies. A no-policy baseline provides a valuable point of comparison to the policy scenarios. The policies included in this scenario represent the policies in place by the end of 2021: state-level Renewable Portfolio Standards (RPS), the Cross-State Air Pollution Rule (CSAPR), the California cap and trade program, and the federal Investment Tax Credit (ITC). Note that these existing policies are also included in the other scenarios described below. In this No Policy scenario, exogenous fuel prices are set based on the [EIA Annual Energy Outlook (AEO) 2022 Reference case](https://www.eia.gov/outlooks/aeo/).
It is worth noting that in August of 2022, the U.S. Congress passed the Inflation Reduction Act (IRA), which allocated ~ $385 billion in funding for climate mitigation activities between 2022 and 2031. This version of the database does not include an explicit analysis of the IRA provisions. We do, however, explore how the provisions in the IRA align with the results of the scenarios included in our modeling efforts and their associated results in the [OEO 2022 report.](https://www.cmu.edu/energy/key-initiatives/open-energy-outlook/oeo-report-2022.html)

### State-level action in the absence of federal policy (‘State Action’)
This scenario considers the possibility of ambitious state-level action to reduce CO<sub>2</sub> emissions without any new federal policy beyond what was available by the end of 2021. The scenario assumes that a collection of U.S. states will implement legislation to achieve net-zero CO<sub>2</sub> emissions from the energy system by 2050. To select the states that would most likely pursue such a policy, we include those states with a renewable or clean energy policy and that have demonstrated a potential willingness to pursue more ambitious actions (e.g., voting history). Figure 1 shows the states with net-zero targets for the energy system in this State Action scenario. This scenario is not meant to serve as a policy forecast but rather an ambitious yet plausible scenario where a subset of state governments take action to reduce emissions. The results inform the degree to which bottom-up, state-level action can reduce emissions compared to the scenarios involving federal action. In this scenario, exogenous fuel prices are set based on the [EIA AEO 2022 Low Oil Price case](https://www.eia.gov/outlooks/aeo/), consistent with expected price impacts due to reductions in petroleum demand.

<img src="documentation_images/StateActionPolicyMap.png" width="60%"/>

**Figure 1.**  States with targets of net-zero CO<sub>2</sub> emissions from the energy system in the State Action scenario

### UNFCCC COP26 commitments (‘COP26’)
This scenario includes the policy commitments in the U.S. Nationally Determined Contributions (NDC), submitted to the UNFCCC as part of the COP26 negotiations. Compared to the No Policy scenario, the results from this scenario indicate how international commitments to climate mitigation can drive reductions in greenhouse gas emissions. Compared with the Net Zero scenario, this COP26 scenario shows the ambition gap between existing international commitments and achieving net-zero emissions. In this scenario, exogenous fuel prices are set based on the [EIA AEO 2022 Low Oil Price case](https://www.eia.gov/outlooks/aeo/).

### Policy neutral net-zero (‘Net Zero’)
This scenario assumes that the United States will reach net-zero CO<sub>2</sub> emissions by 2050. A constraint caps CO<sub>2</sub> emissions across the energy system to achieve this objective, with linear declines beginning in the 2025 model period and reaching net-zero by 2050. The ‘net’ term indicates that the model can balance any residual CO<sub>2</sub> emissions with carbon dioxide removal (CDR) technologies that draw CO<sub>2</sub> directly out of the atmosphere, including biomass integrated gasification combined cycle with carbon capture and sequestration (BECCS) and direct air capture (DAC). The results from this scenario provide a prescriptive look at the energy system transformation to net-zero without regard to the specific policy mechanisms required to achieve it. In this scenario, exogenous fuel prices are set based on the [EIA AEO 2022 Low Oil Price case](https://www.eia.gov/outlooks/aeo/).

### A note on fuel prices
Fuel prices are an exogenous input in this analysis. Specifically, this analysis relies on fuel prices reported in the [US EIA Annual Energy Outlook 2022](https://www.eia.gov/outlooks/aeo/). The No Policy scenario relies on the prices in EIA’s Reference case, while the State Action, COP26, and Net Zero scenarios rely on the prices in EIA’s Low Oil Price case. These differences in the prices among the scenarios aim to capture the price elasticity of supply: as demand for fuels decreases in the State Action, COP26, and Net Zero scenarios, the price also drops.

### Summary 

The scenarios in the report collectively represent a full range of emissions pathways. The No Policy and Net Zero scenarios form upper and lower bounds on the emissions trajectories. The remaining scenarios - State Action and COP26 - represent varying levels of policy ambition that will produce emissions trajectories within the prescribed range.